In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras import layers
import tensorflow_probability as tfp

In [27]:
@tf.function
def Sampling(inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):

    def __init__(self, latent_dim=32, intermediate_dim=64):
        super(Encoder, self).__init__()
        self.dense = layers.Dense(intermediate_dim, activation="tanh",input_shape=(1,))
        self.mean = layers.Dense(latent_dim, activation="linear")
        self.log_var = layers.Dense(latent_dim, activation="linear")

    def call(self, inputs):
        x = self.dense(inputs)
        z_mean = self.mean(x)
        z_log_var = self.log_var(x)
        z = Sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):

    def __init__(self, original_dim, intermediate_dim=64, latent_dim=2):
        super(Decoder, self).__init__()
        self.dense = layers.Dense(intermediate_dim, activation="tanh", input_shape=(1,))
        self.out = layers.Dense(original_dim, activation="linear")

    def call(self, inputs):
        x = self.dense(inputs)
        x = self.out(x)
        return x


class VAE(tf.keras.Model):

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=2
    ):
        super(VAE, self).__init__()
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)

        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed



In [28]:
#x_train = tf.expand_dims(y, axis=1)

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

oirignal_dim = 784
model = VAE(oirignal_dim, 16, 2)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()
train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(16)

epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = model(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(model.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        loss_metric(loss)
    print("epoch %d: mean loss = %.4f" % (epoch, loss_metric.result()))

epoch 0: mean loss = 0.0701
epoch 1: mean loss = 0.0687
